# Deep Learning Project: Adversarial Attacks

This notebook implements and evaluates adversarial attacks (FGSM, PGD) on a ResNet-34 model using a subset of ImageNet. It is converted from a Python script.

In [7]:
import os
import json
import shutil
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
from tqdm import tqdm

# Magic command for matplotlib in notebooks
%matplotlib inline

In [3]:
!pwd

/content


In [4]:
# rm -rf TestDataSet/

In [6]:
from zipfile import ZipFile
import os

zip_path = '/content/TestDataSet.zip'
out_dir  = '/content'

os.makedirs(out_dir, exist_ok=True)
with ZipFile(zip_path, 'r') as z:
    z.extractall(out_dir)

# verify
print(os.listdir(out_dir))

['.config', 'TestDataSet', 'TestDataSet.zip', '__MACOSX', 'sample_data']


In [9]:


# 1) set up transforms exactly as in your spec
mean_norms = np.array([0.485, 0.456, 0.406])
std_norms  = np.array([0.229, 0.224, 0.225])
plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norms, std=std_norms)
])

# 2) point to your dataset root
#    (in Colab this is usually /content/TestDataSet if you unzipped there)
dataset_path = "/content/TestDataSet"

# 3) load labels_list.json and parse out the numeric label indices
with open(os.path.join(dataset_path, "labels_list.json"), "r") as f:
    raw = json.load(f)               # e.g. ["401: accordion", "402: acoustic guitar", …]

label_idxs = []
for entry in raw:
    idx_str, _ = entry.split(":", 1)
    label_idxs.append(int(idx_str.strip()))

# 4) discover your class‐folders in sorted order and map each folder → its numeric label
folders = sorted(
    d for d in os.listdir(dataset_path)
    if os.path.isdir(os.path.join(dataset_path, d))
)
assert len(folders) == len(label_idxs), "Mismatch: #folders vs #labels in JSON"
folder_to_label = {folders[i]: label_idxs[i] for i in range(len(folders))}

# 5) build the ImageFolder & DataLoader
dataset = datasets.ImageFolder(root=dataset_path, transform=plain_transforms)
loader  = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)

# 6) load pretrained ResNet-34
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = models.ResNet34_Weights.IMAGENET1K_V1
model   = models.resnet34(weights=weights).to(device).eval()

# 7) loop & accumulate top-1 / top-5
correct1 = 0
correct5 = 0
total    = 0

with torch.no_grad():
    for images, folder_idxs in tqdm(loader):
        images = images.to(device)
        outputs = model(images)                     # (B,1000)
        _, preds = outputs.topk(5, dim=1)           # (B,5)
        preds = preds.cpu().numpy()
        folder_idxs = folder_idxs.numpy()

        for i, fidx in enumerate(folder_idxs):
            folder_name = dataset.classes[fidx]     # synset e.g. "n02672831"
            gt = folder_to_label[folder_name]       # e.g. 401
            top5 = preds[i].tolist()

            if gt == top5[0]:
                correct1 += 1
            if gt in top5:
                correct5 += 1
            total += 1

# 8) report
print(f"\n\nTop-1 Accuracy: {correct1/total:.4f}")
print(f"Top-5 Accuracy: {correct5/total:.4f}")


100%|██████████| 16/16 [00:00<00:00, 29.04it/s]



Top-1 Accuracy: 0.7600
Top-5 Accuracy: 0.9420
